In [1]:
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision import transforms
from ipywidgets import interact, IntSlider, fixed
from IPython.display import display
from PIL import Image
from skimage.transform import resize
from keras.preprocessing import image
# Assuming utils, evaluation, and explanations modules are custom and present in your project
from utils import *
from evaluation import CausalMetric, auc, gkern
from explanations import RISE
import shap 
cudnn.benchmark = True
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
import json
import torch
from torchvision.models import resnet50
from torch.nn.functional import softmax
from PIL import Image
from torchvision import transforms
from morphology import *


class Morphology(nn.Module):
    '''
    Base class for morpholigical operators 
    For now, only supports stride=1, dilation=1, kernel_size H==W, and padding='same'.
    '''
    def __init__(self, in_channels, out_channels, kernel_size=5, soft_max=True, beta=15, type=None):
        '''
        in_channels: scalar
        out_channels: scalar, the number of the morphological neure. 
        kernel_size: scalar, the spatial size of the morphological neure.
        soft_max: bool, using the soft max rather the torch.max(), ref: Dense Morphological Networks: An Universal Function Approximator (Mondal et al. (2019)).
        beta: scalar, used by soft_max.
        type: str, dilation2d or erosion2d.
        '''
        super(Morphology, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.soft_max = soft_max
        self.beta = beta
        self.type = type

        self.weight = nn.Parameter(torch.zeros(out_channels, in_channels, kernel_size, kernel_size), requires_grad=True)
        self.unfold = nn.Unfold(kernel_size, dilation=1, padding=0, stride=1)

    def forward(self, x):
        '''
        x: tensor of shape (B,C,H,W)
        '''
        # padding
        x = fixed_padding(x, self.kernel_size, dilation=1)
        
        # unfold
        x = self.unfold(x)  # (B, Cin*kH*kW, L), where L is the numbers of patches
        x = x.unsqueeze(1)  # (B, 1, Cin*kH*kW, L)
        L = x.size(-1)
        L_sqrt = int(math.sqrt(L))

        # erosion
        weight = self.weight.view(self.out_channels, -1) # (Cout, Cin*kH*kW)
        weight = weight.unsqueeze(0).unsqueeze(-1)  # (1, Cout, Cin*kH*kW, 1)

        if self.type == 'erosion2d':
            x = weight - x # (B, Cout, Cin*kH*kW, L)
        elif self.type == 'dilation2d':
            x = weight + x # (B, Cout, Cin*kH*kW, L)
        else:
            raise ValueError
        
        if not self.soft_max:
            x, _ = torch.max(x, dim=2, keepdim=False) # (B, Cout, L)
        else:
            x = torch.logsumexp(x*self.beta, dim=2, keepdim=False) / self.beta # (B, Cout, L)

        if self.type == 'erosion2d':
            x = -1 * x

        # instead of fold, we use view to avoid copy
        x = x.view(-1, self.out_channels, L_sqrt, L_sqrt)  # (B, Cout, L/2, L/2)

        return x 

class Dilation2d(Morphology):
    def __init__(self, in_channels, out_channels, kernel_size=5, soft_max=True, beta=20):
        super(Dilation2d, self).__init__(in_channels, out_channels, kernel_size, soft_max, beta, 'dilation2d')

class Erosion2d(Morphology):
    def __init__(self, in_channels, out_channels, kernel_size=5, soft_max=True, beta=20):
        super(Erosion2d, self).__init__(in_channels, out_channels, kernel_size, soft_max, beta, 'erosion2d')



def fixed_padding(inputs, kernel_size, dilation):
    kernel_size_effective = kernel_size + (kernel_size - 1) * (dilation - 1)
    pad_total = kernel_size_effective - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg
    padded_inputs = F.pad(inputs, (pad_beg, pad_end, pad_beg, pad_end))
    return padded_inputs
    
    
if __name__ == '__main__':
    # test
    x=torch.randn(2,3,16,16)
    e=Erosion2d(3, 4, 3, soft_max=False)
    y=e(x)

import tensorflow as tf

# getting ImageNet 1000 class names
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
with open(shap.datasets.cache(url)) as file:
    class_names = [v[1] for v in json.load(file).values()]
    
img_path = 'goldfish.jpg'
img = read_tensor(img_path)

def predict_with_resnet50_pytorch(image_tensor, target_class=1):
    # Ensure the model is in evaluation mode
    model = resnet50(pretrained=True)
    model = model.eval()
    
    # Ensure the tensor is in the correct shape [C, H, W] with C=3, H=W=224
    # Check for batch dimension and add if missing
    if image_tensor.dim() == 3:
        image_tensor = image_tensor.unsqueeze(0)
    
    # Perform prediction
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = F.softmax(output, dim=1)
        percentage = probabilities[0][target_class].item() * 100
        top_probabilities, top_classes = torch.topk(probabilities, k=3)
    # Convert output probabilities to softmax probabilities
    
    # Decode predictions to find the top 3 predictions
    return f'Prediction for class {target_class} = {percentage:.2f}%', top_probabilities[0], top_classes[0]

predicted_classes, top_probabilities, top_classes = predict_with_resnet50_pytorch(img)

# Load black box model for explanations
model = models.resnet50(True)
model = nn.Sequential(model, nn.Softmax(dim=1))
model = model.eval()
model = model.cuda()

for p in model.parameters():
    p.requires_grad = False

model = nn.DataParallel(model)
# Assuming get_class_name is a function to get class names from class indices
# You'll need to define or import this function
print(predicted_classes)
for i in range(len(top_classes)):
    class_name = get_class_name(top_classes[i].item())
    percentage = top_probabilities[i].item() * 100
    print(f'Top {i+1} class: {class_name}, Probability: {percentage:.2f}%')
    
print(predicted_classes)
get_class_name(top_classes[0].item())
target_class = top_classes[0].item()

c:\Users\eagua\anaconda3\envs\torch_cuda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\eagua\anaconda3\envs\torch_cuda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\eagua\anaconda3\envs\torch_cuda\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Prediction for class 1 = 95.19%
Top 1 class: goldfish, Probability: 95.19%
Top 2 class: paintbrush, Probability: 1.32%
Top 3 class: bubble, Probability: 0.70%
Prediction for class 1 = 95.19%


In [2]:
class RISE:
    def __init__(self, model, input_size):
        self.model = model
        self.input_size = input_size

    def generate_masks(self, N, s, p1):
        cell_size = np.ceil(np.array(self.input_size) / s)
        up_size = (s + 1) * cell_size

        grid = np.random.rand(N, s, s) < p1
        grid = grid.astype('float32')

        self.masks = np.empty((N, *self.input_size))

        for i in tqdm(range(N), desc='Generating filters'):
            x = np.random.randint(0, cell_size[0])
            y = np.random.randint(0, cell_size[1])
            self.masks[i, :, :] = resize(grid[i], up_size, order=1, mode='reflect',
                                         anti_aliasing=False)[x:x + self.input_size[0], y:y + self.input_size[1]]
        self.masks = self.masks.reshape(-1, 1, *self.input_size)
        self.masks = torch.from_numpy(self.masks).float().cuda()
        self.N = N
        self.p1 = p1
        print(self.masks.shape)
        return self.masks

    def load_masks(self, filepath):
        self.masks = np.load(filepath)
        self.masks = torch.from_numpy(self.masks).float().cuda()
        self.N = self.masks.shape[0]

explainer = RISE(model, (224, 224))
mask = explainer.generate_masks(100, 8, 0.1)

Generating filters: 100%|██████████| 100/100 [00:00<00:00, 916.61it/s]

torch.Size([100, 1, 224, 224])


In [3]:
img = img.cuda()
erosion = Erosion2d(1, 1, kernel_size=3, soft_max=False, beta=15).to(img.device)
mask = erosion(mask)
masked_image = torch.mul(mask, img)

def visualize_single_mutant(mutants, index):
    """
    Visualize a single mutant image with its score.
    
    Parameters:
    - mutants: Tensor of mutant images.
    - scores: List or array of scores corresponding to the mutant images.
    - index: Index of the mutant image to visualize.
    """
    mutant_image = mutants[index].detach().squeeze(0).cpu().numpy().transpose(1, 2, 0)  # Squeeze and permute dimensions
    plt.figure(figsize=(5, 5))
    plt.imshow(mutant_image)
    plt.axis('off')
    plt.show()

# Example tensor and scores (replace these with your actual data)
num_mutants = img.shape[0]
interact(visualize_single_mutant, 
         mutants=fixed(masked_image), 

         index=IntSlider(min=0, max=num_mutants-1, step=1, description='Mutant Index:'));


interactive(children=(IntSlider(value=0, description='Mutant Index:', max=0), Output()), _dom_classes=('widget…

In [7]:
import inspect
import nnMorpho

def list_package_functions(package):
    functions_list = inspect.getmembers(package, inspect.isfunction)
    for function in functions_list:
        print(function[0])

# List functions directly in nnMorpho package
print("Functions in nnMorpho package:")
list_package_functions(nnMorpho)

# If nnMorpho has submodules, list functions in each submodule
def list_functions_in_submodules(package):
    submodules = [module for module in dir(package) if not module.startswith('__')]
    for submodule_name in submodules:
        submodule = getattr(package, submodule_name)
        if inspect.ismodule(submodule):
            print(f"\nFunctions in {submodule_name} submodule:")
            list_package_functions(submodule)

list_functions_in_submodules(nnMorpho)

'''
def erosion(input_tensor: torch.Tensor,
            structuring_element: torch.Tensor,
            origin: Optional[Union[tuple, List[int]]] = None,
            border: str = 'e',
            block_shape: Tuple[int, int] = (32, 32)):
    """ Erosion is one of the basic operations of Mathematical Morphology. This function computes the grayscale
        erosion of an input tensor by a structuring element.

        Parameters
        ----------
        :param input_tensor: torch.Tensor
            The input tensor to erode. It should be a 2D PyTorch tensor.
        :param structuring_element: torch.Tensor
            The structuring element to erode. The structuring element should be a 2D PyTorch tensor.
        :param origin: None, tuple, List[int]
            The origin of the structuring element. Default to center of the structuring element.
            Negative indexes are allowed.
        :param border: int, float, str
            The value used to pad the image in the border. Two options are allowed when a string is passed in parameter:
            - [a string starting by 'e']: extends naturally the image setting minus infinite value to the border. ('e'
            stands for Euclidean)
            - [any other value]: only takes into account for taking the minimum the values within the input.
            Default value is 'e'.
        :param block_shape: size of the blocks for GPU computation

        Outputs
        -------
        :return: torch.Tensor
            The erosion as a PyTorch tensor of the same shape of the original input.
    """
    # Adapt origin
    if origin is None:
        origin = (structuring_element.shape[0] // 2, structuring_element.shape[1] // 2)

    # Compute erosion
    result = erosion_cpp(input_tensor, structuring_element, origin[0], origin[1], border[0], block_shape[1], block_shape[0])

    return result
'''

Functions in nnMorpho package:


'\ndef erosion(input_tensor: torch.Tensor,\n            structuring_element: torch.Tensor,\n            origin: Optional[Union[tuple, List[int]]] = None,\n            border: str = \'e\',\n            block_shape: Tuple[int, int] = (32, 32)):\n    """ Erosion is one of the basic operations of Mathematical Morphology. This function computes the grayscale\n        erosion of an input tensor by a structuring element.\n\n        Parameters\n        ----------\n        :param input_tensor: torch.Tensor\n            The input tensor to erode. It should be a 2D PyTorch tensor.\n        :param structuring_element: torch.Tensor\n            The structuring element to erode. The structuring element should be a 2D PyTorch tensor.\n        :param origin: None, tuple, List[int]\n            The origin of the structuring element. Default to center of the structuring element.\n            Negative indexes are allowed.\n        :param border: int, float, str\n            The value used to pad the imag